
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Uso-de-psycopg2-para-PostgreSQL" data-toc-modified-id="Uso-de-psycopg2-para-PostgreSQL-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Uso de <code>psycopg2</code> para PostgreSQL</a></span><ul class="toc-item"><li><span><a href="#Conexión-a-la-BBDD" data-toc-modified-id="Conexión-a-la-BBDD-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Conexión a la BBDD</a></span></li><li><span><a href="#Ejecutar-Consultas" data-toc-modified-id="Ejecutar-Consultas-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Ejecutar Consultas</a></span><ul class="toc-item"><li><span><a href="#Ejecutar-Consultas-de-Lectura" data-toc-modified-id="Ejecutar-Consultas-de-Lectura-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Ejecutar Consultas de Lectura</a></span></li><li><span><a href="#Ejecutar-Consultas-de-CREATE-e-INSERT" data-toc-modified-id="Ejecutar-Consultas-de-CREATE-e-INSERT-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Ejecutar Consultas de CREATE e INSERT</a></span></li></ul></li></ul></li><li><span><a href="#Subconsultas" data-toc-modified-id="Subconsultas-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Subconsultas</a></span><ul class="toc-item"><li><span><a href="#Subconsultas-en-el-WHERE" data-toc-modified-id="Subconsultas-en-el-WHERE-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Subconsultas en el <code>WHERE</code></a></span></li><li><span><a href="#Subconsultas-en-el-SELECT" data-toc-modified-id="Subconsultas-en-el-SELECT-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Subconsultas en el <code>SELECT</code></a></span></li><li><span><a href="#Subconsultas-en-el-FROM" data-toc-modified-id="Subconsultas-en-el-FROM-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Subconsultas en el <code>FROM</code></a></span></li></ul></li></ul></div>


# Uso de `psycopg2` para PostgreSQL

`psycopg2` es una librería de Python que nos va a permitir conectar y manejar bases de datos PostgreSQL. Es una de las interfaces más utilizadas para trabajar con PostgreSQL en Python debido a su robustez y facilidad de uso. 

## Conexión a la BBDD

El primer paso para trabajar con `psycopg2` es establecer una conexión a la base de datos. Para eso necesitaremos conocer la información de conexión, como el nombre de la base de datos, usuario, contraseña y el host. Para conectarse a una base de datos PostgreSQL, necesitaremos usasr el método `connect()`. Este método es la puerta de entrada para establecer una conexión con una base de datos PostgreSQL. Este método devuelve un objeto de conexión, que se puede usar para crear un cursor, ejecutar consultas, manejar transacciones, entre otras operaciones. Su sintaxis básica es:

```python
psycopg2.connect(
    database=None, 
    user=None, 
    password=None, 
    host=None, 
    port=None
)
```

Donde:

- **`database`:** Especifica el nombre de la base de datos a la que nos queremos conectar.

- **`user`:** Nombre de usuario con el que nos queremos conectar a la base de datos.

- **`password`:** La contraseña del usuario especificado en el parámetro `user`.

- **`host`:** El servidor donde se encuentra la base de datos. Puede ser `"localhost"` para bases de datos locales o una dirección IP.

- **`port`:** El puerto de la base de datos PostgreSQL. Por defecto es `5432`.

In [1]:
# antes de empezar a trabajar e importar las librerías debemos instalar las siguientes librerías
#!pip install psycopg2
#!pip install psycopg2-binary

In [2]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

# Importar nuestras funciones
# -----------------------------------------------------------------------
from src import soporte_sqlpython as ss

In [3]:
# creamos la conexión con la base de datos de Chinook
connection = psycopg2.connect(
    database="Chinook", # la base de datos con la que queremos trabajar
    user="my_user", # el usuario que tenemos 
    password="admin",
    host="localhost",
    port="5432"
)

In [4]:
# si bien es cierto que crear la conexión es sencillo, es importante controlar los posibles errores que pudieran existir cuando realicemos estas conexiones
try:
    connection = psycopg2.connect(
        database="Chinook", # la base de datos con la que queremos trabajar
        user="my_user", # el usuario que tenemos 
        password="admin",
        host="localhost",
        port="5432")
    print("conexion realizada con éxito")

except OperationalError as e:
    # Manejo de errores comunes
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("Contraseña inválida.")
        
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión.")
    else:
        print(f"Ocurrió un error: {e}")
finally:
    # cerramos la conexion
    connection.close()
    print("conexion cerrada")

conexion realizada con éxito
conexion cerrada


## Ejecutar Consultas

Una vez que hemos establecido una conexión a la base de datos usando `psycopg2`, el siguiente paso es interactuar con los datos mediante consultas. Esta interacción se realiza utilizando un **cursor**, que es un objeto que te permite ejecutar consultas, recuperar resultados y manejar transacciones.


**¿Qué es un *cursor***

Un cursor en `psycopg2` es un objeto que actúa como un controlador para la ejecución de consultas SQL. Es responsable de:

- Ejecutar instrucciones SQL (como `SELECT`, `INSERT`, `UPDATE`, `DELETE`, etc.).

- Recuperar resultados de una consulta.

- Navegar por los registros recuperados (si es necesario).

- Manejar la comunicación entre Python y la base de datos PostgreSQL.

Para crear un cursor, se utiliza el método `cursor()` del objeto de conexión. Su sintaxis es:

```python
# creamos la conexión con la base de datos de Chinook
conexion = psycopg2.connect(
    database="Chinook", # la base de datos con la que queremos trabajar
    user="my_user", # el usuario que tenemos 
    password="admin",
    host="localhost",
    port="5432"
)

cursor = conexion.cursor()
```

En `psycopg2`, las consultas SQL se pueden clasificar en dos grandes categorías:

1. **Consultas de Lectura (SELECT):** Para recuperar datos.

2. **Consultas de Modificación (INSERT, UPDATE, DELETE):** Para modificar o insertar datos en la base de datos.

### Ejecutar Consultas de Lectura

Como ya sabemos una consulta `SELECT` la usaremos para recuperar datos de la base de datos. Supongamos que tenemos una tabla llamada `clientes` con columnas `id`, `nombre`, y `edad`. Tendremos que: 

- Crear la conexión, como hemos aprendido en el apartado anterior. 

- Crear el cursor el cual nos va a permitir controlar la ejecución de las consultas. 

- Ejecutar la query con el método `.execute()` de la librería. 

- Obtener los resultados usando el método `.fetchall()`. Cada registro obtenido nos lo va a devolver en formato de tupla. Si teneoms más de un resultado, nos devolverá una lista de tuplas. 

- Acceder a los resultados obtenidos. Se puede recorrer cada registro y acceder a sus columnas por índice, ya que `fetchall()` devuelve una lista de tuplas.


Hemos hablado de `.fetchall()` para obtener los resultados, pero tenemos dos métodos más que nos pueden ayudar:

- Método `fetchone()`: Recupera **un solo registro** de los resultados de una consulta. Cada vez que llamamos a `fetchone()`, el cursor avanza al siguiente registro disponible. Si no quedan más registros, el método devuelve `None`.


- Método `fetchmany(size)`: Recupera un número determinado de registros basado en el argumento `size`. Este método es útil para manejar grandes conjuntos de resultados sin cargarlos todos en memoria al mismo tiempo. Igual que `.fetchall()`, devuelve una lista de tuplas, donde cada tupla representa un registro.


In [5]:
# creemos una query sencilla para ver como podemos ejecutar nuestras queries y obtener los resultados 
# comom vamos a seguir trabajando con la base de datos de Chinook, vamos a extraer todos los registros de la tabla de 'Genre'

# Conectar a la base de datos
try:
    connection = psycopg2.connect(
        database="Chinook", # la base de datos con la que queremos trabajar
        user="my_user", # el usuario que tenemos 
        password="admin",
        host="localhost")
    print("conexion realizada con éxito")

except OperationalError as e:
    # Manejo de errores comunes
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("Contraseña inválida.")
        
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión.")
    else:
        print(f"Ocurrió un error: {e}")


# Una vez que hemos creado la conexión, creamos un cursor
cursor = connection.cursor()

# Ejecutar una consulta SELECT
cursor.execute('SELECT * FROM "Genre";')

# Recuperar los resultados
resultados = cursor.fetchall()

print("Los resultados son:")
# Procesar los resultados
for registro in resultados:
    print(f'El id es {registro[0]}, y pertenece al género {registro[1]}')

# Cerrar cursor y conexión
cursor.close()
connection.close()

conexion realizada con éxito
Los resultados son:
El id es 1, y pertenece al género Rock
El id es 2, y pertenece al género Jazz
El id es 3, y pertenece al género Metal
El id es 4, y pertenece al género Alternative & Punk
El id es 5, y pertenece al género Rock And Roll
El id es 6, y pertenece al género Blues
El id es 7, y pertenece al género Latin
El id es 8, y pertenece al género Reggae
El id es 9, y pertenece al género Pop
El id es 10, y pertenece al género Soundtrack
El id es 11, y pertenece al género Bossa Nova
El id es 12, y pertenece al género Easy Listening
El id es 13, y pertenece al género Heavy Metal
El id es 14, y pertenece al género R&B/Soul
El id es 15, y pertenece al género Electronica/Dance
El id es 16, y pertenece al género World
El id es 17, y pertenece al género Hip Hop/Rap
El id es 18, y pertenece al género Science Fiction
El id es 19, y pertenece al género TV Shows
El id es 20, y pertenece al género Sci Fi & Fantasy
El id es 21, y pertenece al género Drama
El id es 22

### Ejecutar Consultas de CREATE e INSERT

Como ya aprendimos en las lecciones anteriores, para crear e insertar usaremos las sentencias CREATE e INSERT:

- **`CREATE TABLE`**: Este comando se utiliza para crear una nueva tabla en la base de datos, definiendo las columnas y sus tipos de datos.

- **`INSERT INTO`**: Este comando se utiliza para agregar nuevos registros a una tabla ya existente.

Los pasos que deberemos dar a la hora de crear e insertar datos son: 

- Crear la conexión. 

- Crear el cursor, el cual nos va a permitir controlar la ejecución de las consultas. 

- Ejecutar la consulta con el método `.execute()`.

- Confirmar la creación o inserción usando el método `.commit()`. 

Para entender todo esto vamos a crear una tabla sencilla llamada  `clientes` con tres columnas: `id`, `nombre`, y `edad`.


In [6]:
try:
    # Conectar a la base de datos
    connection = psycopg2.connect(
        host="localhost",
        database="aprendiendo",
        user="my_user",
        password="admin"
    )
    cursor = connection.cursor()

    # Crear la tabla si no existe
    query_create_table = """
    CREATE TABLE IF NOT EXISTS clientes (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(100),
        edad INT
    );
    """
    cursor.execute(query_create_table)
    connection.commit()
    print("Tabla 'clientes' creada exitosamente")

    # Insertar un registro en la tabla
    query_insert = "INSERT INTO clientes (nombre, edad) VALUES (%s, %s);"
    valores = ("Carlos Pérez", 35)
    cursor.execute(query_insert, valores)
    connection.commit()
    print("Registro insertado exitosamente")

except Exception as error:
    print(f"Ocurrió un error: {error}")
    connection.rollback() # revierte cualquier cambio que se haya realizado en la base de datos durante la transacción actual, si ocurre un error. En otras palabras, deshace cualquier modificación que aún no haya sido confirmada con commit().

finally:
    # Cerrar cursor y conexión
    if cursor:
        cursor.close()
    if connection:
        connection.close()
    print("Conexión cerrada")

Tabla 'clientes' creada exitosamente
Registro insertado exitosamente
Conexión cerrada


El ejemplo que acabamos de ver es sencillo, pero no real, ya que normalmente cuando trabajamos con bases de datos, normalmente queremos insertar más de un registro. Para eso necesitaremos usar el método `.executemany()`. Su sintaxis básica es: 

```python
cursor.executemany(query, datos)
```
Donde: 

- **`query`:** Es la consulta SQL que se va a ejecutar repetidamente. Esta consulta debe incluir placeholders (`%s`) para cada valor que será dinámico.

- **`datos`:** Es una secuencia (por ejemplo, una lista de tuplas) donde cada elemento contiene los valores que se pasarán a la consulta en cada ejecución.


In [7]:
try:
    # Conectar a la base de datos
    connection = psycopg2.connect(
        host="localhost",
        database="aprendiendo",
        user="my_user",
        password="admin"
    )
    cursor = connection.cursor()


    # Crear la tabla si no existe
    query_create_table = """
    CREATE TABLE IF NOT EXISTS clientes (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(100),
        edad INT
    );
    """
    cursor.execute(query_create_table)
    connection.commit()

    # Insertar múltiples registros
    query_insert = "INSERT INTO clientes (nombre, edad) VALUES (%s, %s);"
    valores_multiples = [
        ("Ana García", 28),
        ("Luis Fernández", 40),
        ("María López", 22)
    ]
    cursor.executemany(query_insert, valores_multiples)
    connection.commit()
    print("Múltiples registros insertados exitosamente")

except Exception as error:
    print(f"Ocurrió un error: {error}")
    connection.rollback()

Múltiples registros insertados exitosamente


# Subconsultas

Las subconsultas son consultas anidadas dentro de una consulta principal. Son útiles  porque permiten realizar consultas más complejas y obtener información más específica al combinar y relacionar datos de múltiples tablas.

Las subconsultas se utilizan para realizar consultas en un conjunto de datos más pequeño y luego utilizar esos resultados en la consulta principal. Esto permite filtrar y limitar los resultados de una manera más precisa.

Además, las subconsultas se tienen que utilizar en  partes concretas de las consulta, esto es en las cláusulas SELECT, WHERE o FROM. Las subconsultas deben cumplir una serie de normas:

- Deben estar encerradas entre paréntesis.
- **NO** deben incluir una cláusula `ORDER BY`.
- Deben tener al menos la estructura básica (SELECT expresión FROM tabla).

Las ventajas de usar subconsultas son:

- Las subconsultas permiten dividir una consulta compleja en varias consultas más pequeñas y manejables. Esto hace que sea más fácil de entender y mantener el código.

- Las subconsultas se utilizan para filtrar y limitar los resultados de una consulta principal. Esto permite obtener información más específica y precisa.

- Las subconsultas pueden realizar cálculos y operaciones adicionales en los resultados de una consulta principal. Esto permite realizar análisis más avanzados y obtener información más detallada.

- Las subconsultas se utilizan para combinar y relacionar datos de múltiples tablas en una consulta. Esto facilita la obtención de información de diferentes fuentes y la creación de consultas más completas.

- Las subconsultas brindan mayor flexibilidad y control sobre los resultados de una consulta. Permiten realizar consultas más específicas y adaptadas a las necesidades de análisis de datos.

Como hemos dicho, **las subconsultas se pueden construir únicamente en el SELECT, WHERE, FROM y HAVING.** Veamos en detalle cada una de ellas. 


## Subconsultas en el `WHERE`

Para construir una subconsulta en el `WHERE` deberemos:

1. Encerrar la subconsulta entre paréntesis: Para comenzar a construir una subconsulta en el WHERE, debemos encerrarla entre paréntesis. Esto indica que es una consulta anidada dentro de la consulta principal.

2. Escribe la consulta en el interior de los paréntesis: Dentro de los paréntesis, escribe la consulta que deseas realizar. Puedes utilizar cualquier tipo de consulta válida en PostgreSQL, como una SELECT, UPDATE, DELETE, etc.

3. Utiliza los resultados de la subconsulta en el WHERE: Después de la subconsulta, puedes utilizar los resultados obtenidos en el WHERE para filtrar los resultados de la consulta principal. Puedes utilizar operadores de comparación como "=", "<", ">", etc., para comparar los resultados de la subconsulta con los valores de la consulta principal, pero también podremos usar el `IN` y `NOT IN` que aprendimos en las lecciones del WHERE. 

Su sintaxis básica es:

```sql
SELECT columnas
FROM tabla
WHERE columna IN 
    (SELECT columna FROM otra_tabla WHERE condicion);
```

**Ventajas que tiene usar subconsultas**

- Simplicidad y legibilidad: el uso de las subconsultas es mas simple y fácil de entender, la subconsulta simplifica la lógica.

- Optimización del rendimiento: la subconsulta puede mejorar el rendimiento de la query, es decir, su ejecución puede ser más rápida.

- Flexibilidad: Las subconsultas permiten una mayor flexibilidad al permitir condiciones más complejas y cálculos en la sentencia `WHERE`.

In [8]:
# Imaginemos que queremos seleccionar todas las facturas cuyo total sea mayor al promedio de todos los totales de facturas en la tabla. 
# Si no estuviéramos trabajando en SQL lo que haríamos sería sacar la media de las facturas y luego filtrar para quedarnos solo con aquellas cuyo total sea mayor que esa media. 
# Podríamos pensar que en el SQL lo podríamos hacer directamente en el `WHERE`. 

query_subconsulta1 = '''SELECT * 
                            FROM "Invoice" AS i 
                            WHERE "Total" > AVG("Total"); '''
resultado_subconsulta1 = ss.extraer_datos("Chinook", "my_user", "admin", query_subconsulta1)

# Leyendo el error vemos que nos dice que no podemos incluir funciones de agregación en el `WHERE` , podríamos pensar en un `HAVING`,
# pero recordemos que esta función debe ir asociada a un `GROUP BY`.  Por lo que con lo que sabemos hasta ahora no nos quedaría otra más que hacerlo en dos pasos, 
# primero sacar la media y luego filtrar con ese valor

Conexión realizada con éxito


GroupingError: aggregate functions are not allowed in WHERE
LINE 3:                             WHERE "Total" > AVG("Total"); 
                                                    ^


Como hemos dicho, las subqueries están divididas en 2 queries, la query principal y la subquery que la tendremos que poner entre paréntesis. En nuestro caso:

- La subquery será la query que nos permita calcular la media de la columna `Total`.

- La query principal será la query que filtre los datos con el valor obtenido en la subquery.


In [9]:
# La consulta selecciona el "InvoiceId" y el "Total" de la tabla "Invoice". 
# Luego, filtra los resultados utilizando una subconsulta en el WHERE. 
# La subconsulta calcula el promedio del "Total" de todas las facturas en la tabla 
# "Invoice" y redondea el resultado a dos decimales. A continuación, 
# la consulta principal selecciona todas las facturas cuyo "Total" es mayor que 
# el promedio calculado en la subconsulta. Por último, ordena los resultados en 
# orden ascendente según el "Total".

query_subconsulta2 = '''
                    SELECT "InvoiceId", "Total" FROM "Invoice"
                    WHERE "Total" > 
                                    (SELECT ROUND(AVG("Total"), 2) 
                                        FROM "Invoice" )
                    ORDER BY "Total" ASC ;
                    '''
resultado_subconsulta2 = ss.extraer_datos("Chinook", "my_user", "admin", query_subconsulta2)
df_subconsulta2 = pd.DataFrame(resultado_subconsulta2, columns = ["InvoiceID", "Total"])
df_subconsulta2.head()

Conexión realizada con éxito
Los resultados son:
(374, Decimal('5.94'))


,InvoiceID,Total
0,374,5.94
1,143,5.94
2,45,5.94
3,255,5.94
4,150,5.94


In [10]:
# Pongamos otro ejemplo, en este caso vamos a querer seleccionar todas las canciones que pertenecen a los álbumes de AC/DC.
# Lo primero que tenemos que pensar es que tablas vamos a necesitar para resolver esta pregunta, en concreto necesitaremos la tabla de `Track`,
# `Album` y `Artist` que están conectadas por el `AlbumId`y `ArtistId` respectivamente. 
# SOLUCIÓN CON JOINS
query_subconsulta3_1 = '''SELECT t."Name" AS nombre_cancion , a2."Name" AS nombre_artista, a."Title" AS nombre_album  
                        FROM "Track" AS t 
                        INNER JOIN "Album" AS a 
                            ON t."AlbumId" = a."AlbumId" 
                        INNER JOIN "Artist" AS a2 
                            ON a."ArtistId" = a2."ArtistId" 
                        WHERE a2."Name" = 'AC/DC'
                        ORDER BY nombre_cancion;
                        '''
resultado_subconsulta3_1 = ss.extraer_datos("Chinook", "my_user", "admin", query_subconsulta3_1)
df_subconsulta3_1 = pd.DataFrame(resultado_subconsulta3_1, columns = ["nombre_cancion", "nombre_artista", "nombre_album"])
df_subconsulta3_1.head()


Conexión realizada con éxito
Los resultados son:
('Bad Boy Boogie', 'AC/DC', 'Let There Be Rock')


,nombre_cancion,nombre_artista,nombre_album
0,Bad Boy Boogie,AC/DC,Let There Be Rock
1,Breaking The Rules,AC/DC,For Those About To Rock We Salute You
2,C.O.D.,AC/DC,For Those About To Rock We Salute You
3,Dog Eat Dog,AC/DC,Let There Be Rock
4,Evil Walks,AC/DC,For Those About To Rock We Salute You


In [11]:
# SOLUCIÓN CON SUBCONSULTAS
query_subconsulta3_2 = '''SELECT t."Name" as nombre_cancion
                        FROM "Track" t 
                        WHERE "AlbumId"  IN (
                            SELECT "AlbumId"
                            FROM "Album" a
                            WHERE "ArtistId" = (
                                SELECT "ArtistId"
                                FROM "Artist" a2
                                WHERE "Name"  = 'AC/DC'
                            )
                        )
                        ORDER BY nombre_cancion;
                    '''
resultado_subconsulta3_2 = ss.extraer_datos("Chinook", "my_user", "admin", query_subconsulta3_2)
df_subconsulta3_2 = pd.DataFrame(resultado_subconsulta3_2, columns = ["nombre_cancion"])
df_subconsulta3_2.head()

Conexión realizada con éxito
Los resultados son:
('Bad Boy Boogie',)


,nombre_cancion
0,Bad Boy Boogie
1,Breaking The Rules
2,C.O.D.
3,Dog Eat Dog
4,Evil Walks


<div style="background-color: #7b7d7d; padding: 10px; border-left: 6px solid #000080; color: black; border-radius: 10px;">


**Dividamos la consulta para entenderla mejor:**

- La última de las subqueries devuelve los Id de los Artistas (`ArtistId`)  filtrados en la tabla `Artist`  para quedarse con el Id de AC/DC. Esta consulta selecciona el "ArtistId" de la tabla "Artist" donde el 
    nombre del artista es 'AC/DC'. Esto se logra mediante la cláusula WHERE que 
    filtra las filas de la tabla "Artist" y devuelve solo las filas donde el 
    nombre es 'AC/DC'.
    
    ```sql
    SELECT "ArtistId"
    FROM "Artist" a2
    WHERE "Name"  = 'AC/DC'
    ```
    


- La segunda de las subconsultas selecciona el `AlbumId` y filtra estos id para quedarse solo con aquellos que corresponden con el `ArtistId` coincide con el resultado de la subquery anterior. Esta consulta se utiliza para obtener los id de los álbumes de la tabla "Album" que pertenecen al artista "AC/DC".
    
    ```sql
    SELECT "AlbumId"
    FROM "Album" a
    WHERE "ArtistId" = (
            SELECT "ArtistId"
            FROM "Artist" a2
            WHERE "Name"  = 'AC/DC'
    		    )
    ```

- En esta ultima parte ya no usamos el operador `=` en la subquery sino que usamos el operador `IN`. Esto es así porque la query anterior nos devuelve dos valores, por lo que no podríamos incluir un `=`. Lo que estamos haciendo es seleccionar los `AlbunId` que están contenidos en el resultado del paso anterior. En esta query estamos seleccionando el nombre de las canciones de la tabla "Track" que pertenecen a los álbumes del artista "AC/DC". Utiliza una subconsulta en el WHERE para filtrar los resultados. La subconsulta obtiene los "AlbumId" de la tabla "Album" donde el "ArtistId" coincide con el "ArtistId" obtenido de la tabla "Artist" con el nombre "AC/DC". Luego, la consulta principal selecciona las canciones cuyo "AlbumId" está en la lista de "AlbumId" obtenida de la subconsulta. Por último, ordena los resultados por el nombre de la canción y limitando los resultados a 5. 
    
    ```sql
    SELECT t."Name" as nombre_cancion,  
    FROM "Track" t 
    WHERE "AlbumId"  IN (
        SELECT "AlbumId"
        FROM "Album" a
        WHERE "ArtistId" = (
            SELECT "ArtistId"
            FROM "Artist" a2
            WHERE "Name"  = 'AC/DC'
        )
    )
    ORDER BY nombre_cancion
    LIMIT 5;
    ```
</div>

Por último, dentro de las subqueries en el `WHERE` podemos usar las sentencias `EXISTS` y `NOT EXISTS`. Veámoslas en más detalle: 

- **EXISTS:** Se utiliza para verificar si una subconsulta devuelve algún resultado. Si la subconsulta devuelve al menos una fila, la condición `EXISTS` se evalúa como verdadera y la consulta principal continúa ejecutándose. Su sintaxis básica es:
    
    ```sql
    SELECT columnas FROM tabla_principal WHERE EXISTS (subconsulta);
    ```
    
- **NOT EXISTS:** Es similar a `EXISTS`, pero verifica si la subconsulta no devuelve ningún resultado. Si la subconsulta no devuelve ninguna fila, la condición `NOT EXISTS` se evalúa como verdadera y la consulta principal continúa ejecutándose. Su sintaxis básica es: 
     
    ```sql
    SELECT columnas FROM tabla_principal WHERE NOT EXISTS (subconsulta);
    ```
    


In [12]:
# Supongamos que queremos encontrar todos los clientes que tienen al menos una factura. SOLUCIÓN CON JOINS
query_subconsulta4_1 = '''SELECT CONCAT( c."FirstName" , ' ',  c."LastName") as nombre_cliente, i."InvoiceId"  
                        FROM "Customer" AS c 
                        INNER JOIN "Invoice" AS i 
                            ON c."CustomerId" = i."CustomerId" 
                        ORDER BY nombre_cliente;
                        '''
resultado_subconsulta4_1 = ss.extraer_datos("Chinook", "my_user", "admin", query_subconsulta4_1)
df_subconsulta4_1 = pd.DataFrame(resultado_subconsulta4_1, columns = ["nombre_cliente", "invoice_id"])
df_subconsulta4_1.head()

Conexión realizada con éxito
Los resultados son:
('Aaron Mitchell', 342)


,nombre_cliente,invoice_id
0,Aaron Mitchell,342
1,Aaron Mitchell,116
2,Aaron Mitchell,290
3,Aaron Mitchell,268
4,Aaron Mitchell,245


In [13]:
query_subconsulta4_2 = '''SELECT "CustomerId",
       concat("FirstName", ' ', "LastName") as nombre_cliente
        FROM "Customer" c
        WHERE EXISTS (
            SELECT "CustomerId"
            FROM "Invoice" i
            WHERE i."CustomerId" = c."CustomerId"
        )
        order by nombre_cliente;
'''

resultado_subconsulta4_2 = ss.extraer_datos("Chinook", "my_user", "admin", query_subconsulta4_2)
df_subconsulta4_2 = pd.DataFrame(resultado_subconsulta4_2, columns = ["nombre_cliente", "invoice_id"])
df_subconsulta4_2.head()

Conexión realizada con éxito
Los resultados son:
(32, 'Aaron Mitchell')


,nombre_cliente,invoice_id
0,32,Aaron Mitchell
1,11,Alexandre Rocha
2,7,Astrid Gruber
3,4,Bjørn Hansen
4,39,Camille Bernard


<div style="background-color: #7b7d7d; padding: 10px; border-left: 6px solid #000080; color: black; border-radius: 10px;">


**Dividamos la consulta para entenderla mejor:**

Entendamos la consulta: 

- En el **`SELECT`** y el **`FROM`** estamos seleccionado las columnas que nos interesan de la tabla “Customer.

- **`WHERE EXISTS` (...)**: Esta sentencia es un  condicional que filtra los resultados. La subconsulta dentro de EXISTS busca registros en la tabla "Invoice" que cumplan cierta condición, la condición es la que especificamos en el `WHERE`, es decir, la query irá chequeando si el “CustomerId” de la tabla “Customers” esta en la tabla de “Invoice”.

La primera diferencia que vemos entre los resultados es que al hacer la subconsulta con el `EXISTS` no hay nombres duplicados de los clientes. Lo cual es muy interesante si solo queremos saber que clientes nos han comprado. 

Si por otro lado quisiéramos saber aquellos clientes que no han hecho ninguna compra, lo único que deberíamos hacer es usar la consulta `NOT EXISTS`, que en este caso no nos devuelve ningún resultado, es decir, todos nuestros clientes han hecho al menos una compra. 

</div>

## Subconsultas en el `SELECT`

A veces necesitamos información adicional o realizar cálculos específicos para cada fila devuelta por nuestra consulta principal. Aquí es donde entran en juego las subconsultas en la cláusula `SELECT`. Las subconsultas en la cláusula `SELECT` se colocan dentro de la lista de selección de columnas, entre paréntesis y separadas por comas, después de la palabra clave `SELECT`. Cada subconsulta en la cláusula `SELECT` se evalúa para cada fila devuelta por la consulta principal. Esto significa que la subconsulta se ejecutará múltiples veces, una vez por cada fila en los resultados de la consulta principal.

La sintaxis básica para una subconsulta en el SELECT es la siguiente:

```sql
SELECT 
    columna1,
    (SELECT columna2 FROM tabla2 WHERE condición) AS alias
FROM 
    tabla1;
```

Empecemos con una subquery sencilla, en este caso vamos a calcular el total de facturas por cliente. Para eso necesitaremos las tablas de `Invoice` y `Customer` que están conectadas por la columna `CustomerId`. Solucionemos primero el ejercicio con una query normal y luego con una subquery. Con una query normal tendremos que hacer la siguiente query: 


In [14]:
# En esta query seleccionamos el número de facturas y el nombre completo de cada cliente de las tablas "Invoice" y "Customer". Utiliza un INNER JOIN para combinar 
# las filas de ambas tablas basándose en el "CustomerId". Luego, agrupa los resultados por el "CustomerId" para obtener el número de facturas por cliente. 
# Por último, ordena los resultados en orden ascendente según el número de facturas, y el nombre cliente limitando los resultados a 5 y mostrando los resultados a 
# partir del tercero.
query_subconsulta5_1 = '''SELECT count(*) AS numero_facturas, concat(c."FirstName" , ' ',  c."LastName") AS nombre_cliente 
                                FROM "Invoice" AS i 
                                INNER JOIN "Customer" AS c 
                                ON i."CustomerId" = c."CustomerId"
                                GROUP BY c."CustomerId" 
                                ORDER BY numero_facturas, nombre_cliente;
                                '''
resultado_subconsulta5_1 = ss.extraer_datos("Chinook", "my_user", "admin", query_subconsulta5_1)
df_subconsulta5_1 = pd.DataFrame(resultado_subconsulta5_1, columns = ["numero_facturas", "nombre_cliente"])
df_subconsulta5_1.head()

Conexión realizada con éxito
Los resultados son:
(6, 'Puja Srivastava')


,numero_facturas,nombre_cliente
0,6,Puja Srivastava
1,7,Aaron Mitchell
2,7,Alexandre Rocha
3,7,Astrid Gruber
4,7,Bjørn Hansen


In [15]:
query_subconsulta5_2 = '''SELECT 
                            (
                                SELECT COUNT(*)
                                FROM "Invoice" AS i_sub
                                WHERE i_sub."CustomerId" = c."CustomerId"
                            ) AS numero_facturas,
                            CONCAT(c."FirstName", ' ', c."LastName") AS nombre_cliente
                            FROM 
                                "Customer" AS c
                            ORDER BY 
                                numero_facturas, nombre_cliente;
                        '''
resultado_subconsulta5_2 = ss.extraer_datos("Chinook", "my_user", "admin", query_subconsulta5_2)
df_subconsulta5_2 = pd.DataFrame(resultado_subconsulta5_2, columns = ["numero_facturas", "nombre_cliente"])
df_subconsulta5_2.head()

Conexión realizada con éxito
Los resultados son:
(6, 'Puja Srivastava')


,numero_facturas,nombre_cliente
0,6,Puja Srivastava
1,7,Aaron Mitchell
2,7,Alexandre Rocha
3,7,Astrid Gruber
4,7,Bjørn Hansen


## Subconsultas en el `FROM`

Las subconsultas en la cláusula `FROM` permiten utilizar los resultados de una consulta como si fueran una tabla temporal dentro de otra consulta. Las subconsultas en la cláusula `FROM` se ejecutan antes de la consulta principal. La consulta principal utiliza los resultados de la subconsulta como si fueran una tabla temporal. Además, las subconsultas en la cláusula `FROM` deben tener un alias, que se utiliza para hacer referencia a los resultados de la subconsulta en la consulta principal.

```sql
SELECT columnas
FROM (subconsulta) AS alias
```

Supongamos que queremos saber quienes son los 5 empleados con mas ventas de la compañía. 


In [16]:
query_subconsulta6_1 = '''SELECT concat("FirstName" , ' ',  "LastName") as nombre_empleado , sum("Total") as ventas_totales
                            FROM (
                                SELECT AVG("Total") AS salario_promedio
                                FROM "Invoice" i 
                                    ) AS sp,
                            "Invoice" i2 
                            INNER JOIN "Customer" c 
                                ON c."CustomerId" = i2."CustomerId" 
                            WHERE i2."Total"  > sp.salario_promedio
                            GROUP BY CONCAT("FirstName" , ' ',  "LastName") 
                            ORDER BY ventas_totales
                            LIMIT 5;
                        '''
resultado_subconsulta6_1 = ss.extraer_datos("Chinook", "my_user", "admin", query_subconsulta6_1)
df_subconsulta6_1 = pd.DataFrame(resultado_subconsulta6_1, columns = ["nombre_empleado", "ventas_totales"])
df_subconsulta6_1.head()

Conexión realizada con éxito
Los resultados son:
('Kara Nielsen', Decimal('28.71'))


,nombre_empleado,ventas_totales
0,Kara Nielsen,28.71
1,Heather Leacock,28.71
2,Diego Gutiérrez,28.71
3,Michelle Brooks,28.71
4,François Tremblay,28.71


In [17]:
query_subconsulta6_2 = '''SELECT concat(c."FirstName", ' ', c."LastName"),
                            total_compras
                        FROM "Customer" c
                        INNER JOIN (
                            SELECT "CustomerId", COUNT(*) AS total_compras
                            FROM "Invoice"
                            GROUP BY "CustomerId"
                        ) AS compras_por_cliente ON c."CustomerId" = compras_por_cliente."CustomerId";
                        '''
resultado_subconsulta6_2 = ss.extraer_datos("Chinook", "my_user", "admin", query_subconsulta6_2)
df_subconsulta6_2 = pd.DataFrame(resultado_subconsulta6_2, columns = ["nombre_empleado", "ventas_totales"])
df_subconsulta6_2.head()

Conexión realizada con éxito
Los resultados son:
('Luís Gonçalves', 7)


,nombre_empleado,ventas_totales
0,Luís Gonçalves,7
1,Leonie Köhler,7
2,François Tremblay,7
3,Bjørn Hansen,7
4,Frantiek Wichterlová,7


## Ejercicios para Practicar Subconsultas en Clase

In [23]:
# **Ejercicio 1: Filtrar Álbumes por Artistas con Más de 5 Álbumes**
# Encuentra los nombres de los álbumes de aquellos artistas que tienen más de 5 álbumes en la base de datos.

query_ej1 = '''SELECT a."Title"
                FROM "Album" a
                WHERE a."ArtistId"  IN (
                    SELECT "ArtistId"
                    FROM "Album"
                    GROUP BY "ArtistId"
                    HAVING COUNT("AlbumId") > 5);
                '''
resultado_ej1 = ss.extraer_datos("Chinook", "my_user", "admin", query_ej1)
df_ej1 = pd.DataFrame(resultado_ej1, columns = ["titulo"])
df_ej1.head()

Conexión realizada con éxito
Los resultados son:
('BBC Sessions [Disc 1] [Live]',)


,titulo
0,BBC Sessions [Disc 1] [Live]
1,Garage Inc. (Disc 1)
2,MK III The Final Concerts [Disc 1]
3,Physical Graffiti [Disc 1]
4,The Final Concerts (Disc 2)


In [24]:
#**Ejercicio 2: Listar Pistas con Precio Superior al Promedio**
# Encuentra las pistas (`tracks`) cuyo precio es superior al precio promedio de todas las pistas en la base de datos.


query_ej2 = '''SELECT t."Name" , t."UnitPrice" 
                FROM "Track" t 
                WHERE t."UnitPrice"  > (
                    SELECT AVG("UnitPrice")
                    FROM "Track");
            '''
resultado_ej2 = ss.extraer_datos("Chinook", "my_user", "admin", query_ej2)
df_ej2 = pd.DataFrame(resultado_ej2, columns = ["nombre", "precio_unitario"])
df_ej2.head()

Conexión realizada con éxito
Los resultados son:
('Battlestar Galactica: The Story So Far', Decimal('1.99'))


,nombre,precio_unitario
0,Battlestar Galactica: The Story So Far,1.99
1,Occupation / Precipice,1.99
2,"Exodus, Pt. 1",1.99
3,"Exodus, Pt. 2",1.99
4,Collaborators,1.99


In [26]:
# **Ejercicio 3: Álbumes que Contienen Canciones de un Género Específico**
# Lista los títulos de los álbumes que contienen al menos una pista del género 'Rock'.

query_ej3 = '''SELECT a."Title" 
                FROM "Album" a 
                WHERE a."AlbumId"  IN (
                        SELECT t."AlbumId"  
                        FROM "Track" t 
                        JOIN "Genre" g 
                            ON t."GenreId"  = g."GenreId" 
                        WHERE g."Name" = 'Rock'
                );
            '''
resultado_ej3 = ss.extraer_datos("Chinook", "my_user", "admin", query_ej3)
df_ej3 = pd.DataFrame(resultado_ej3, columns = ["titulo"])
df_ej3.head()

Conexión realizada con éxito
Los resultados son:
('Emergency On Planet Earth',)


,titulo
0,Emergency On Planet Earth
1,In Your Honor [Disc 2]
2,Led Zeppelin I
3,Bark at the Moon (Remastered)
4,Raul Seixas


In [27]:
# **Ejercicio 4: Clientes que Han Comprado un Álbum de un Artista Específico**
# Encuentra los nombres de los clientes que han comprado al menos un álbum del artista 'AC/DC'.

query_ej4 = '''SELECT c."FirstName" , c."LastName" 
                FROM "Customer" c 
                WHERE c."CustomerId"  IN (
                            
                        SELECT i."CustomerId"
                            FROM "Invoice" i
                            JOIN "InvoiceLine" il  ON i."InvoiceId"  = il."InvoiceId"
                            JOIN "Track" t  ON il."TrackId"  = t."TrackId"
                            JOIN "Album" a ON t."AlbumId"  = a."AlbumId"
                            JOIN "Artist" a2  ON a."ArtistId"  = a2."ArtistId"
                            WHERE a2."Name"  = 'AC/DC');
            '''
resultado_ej4 = ss.extraer_datos("Chinook", "my_user", "admin", query_ej4)
df_ej4 = pd.DataFrame(resultado_ej4, columns = ["nombre", "apellido"])
df_ej4.head()


Conexión realizada con éxito
Los resultados son:
('Bjørn', 'Hansen')


,nombre,apellido
0,Bjørn,Hansen
1,Daan,Peeters
2,Fernanda,Ramos
3,Ellie,Sullivan
4,Lucas,Mancini


In [29]:
# **Ejercicio 5: Subquery en el FROM para Encontrar el Número de Pistas por Álbum**
# Encuentra el número de pistas (`tracks`) por cada álbum y lista los álbumes que tienen más de 15 pistas.


query_ej5 = '''SELECT album_info."Title", album_info.TrackCount
                FROM (
                        SELECT a."Title" , COUNT(t."TrackId") AS TrackCount
                            FROM "Album" a
                            JOIN "Track" t  ON a."AlbumId"  = t."AlbumId"
                            GROUP BY a."Title" 
                    
                ) AS album_info
                WHERE album_info.TrackCount > 15;
            '''
resultado_ej5 = ss.extraer_datos("Chinook", "my_user", "admin", query_ej5)
df_ej5 = pd.DataFrame(resultado_ej5, columns = ["titulo", "cuenta"])
df_ej5.head()

Conexión realizada con éxito
Los resultados son:
('The Cream Of Clapton', 18)


,titulo,cuenta
0,The Cream Of Clapton,18
1,Minha Historia,34
2,Chill: Brazil (Disc 2),17
3,"Battlestar Galactica, Season 3",19
4,Instant Karma: The Amnesty International Campa...,23


# Vistas

Las **vistas**  son como ventanas virtuales a los datos almacenados en la base de datos. Funcionan como tablas personalizadas, pero en realidad, no almacenan datos físicamente. En cambio, representan resultados de consultas SQL definidas previamente. Las vistas permiten a los usuarios acceder a datos de manera conveniente y segura, sin necesidad de conocer la complejidad de las consultas subyacentes. Son útiles para simplificar el acceso a los datos y para proporcionar una capa de abstracción sobre las tablas base, lo que facilita la gestión y la seguridad de los datos.

Las ventajas de las vistas son: 

- **Simplificación de consultas**: Permiten definir consultas complejas una vez y utilizarlas repetidamente como si fueran tablas normales, lo que evita que tengamos que estar construyendo queries complejas constantemente. Es como si creáramos una biblioteca de queries. 

- **Seguridad**: Pueden utilizarse para restringir el acceso a ciertas columnas o filas, lo que permite controlar de manera más granular quién puede acceder a qué datos.

- **Optimización de rendimiento**: Al predefinir consultas comunes, las vistas pueden mejorar el rendimiento al evitar la necesidad de ejecutar las mismas consultas repetidamente desde diferentes ubicaciones en la aplicación.

- **Facilidad de mantenimiento**: Al definir consultas una vez en una vista, cualquier cambio en la lógica de la consulta solo necesita realizarse en un solo lugar, lo que simplifica el mantenimiento y reduce la posibilidad de errores.

Para construir una vista deberemos seguir la siguiente sintaxis:

```sql
CREATE VIEW nombre_vista AS 
query que queremos
```


In [19]:
# Vamos a crear una vista donde tengamos los resultados de una query donde tengamos el nombre de la canción y el nombre de los artistas de la BBDD

query_crear_vista = '''CREATE VIEW VistaArtistAlbum2 AS 
                select a2."Title" AS nombre_cancion, a."Name" AS nombre_artista 
                FROM "Artist" a 
                INNER JOIN "Album" a2 
                ON a."ArtistId" = a2."AlbumId";
            '''
ss.vistas_tablas("Chinook", "my_user", "admin", query_crear_vista)

query_llamar_vista = 'SELECT * FROM VistaArtistAlbum2;'
ss.extraer_datos("Chinook", "my_user", "admin", query_llamar_vista)[:3]

Conexión realizada con éxito
Conexión realizada con éxito
Los resultados son:
('For Those About To Rock We Salute You', 'AC/DC')


[('For Those About To Rock We Salute You', 'AC/DC'),
 ('Balls to the Wall', 'Accept'),
 ('Restless and Wild', 'Aerosmith')]

# Tablas Temporales

Las **tablas temporales** son tablas que existen solo durante la sesión de la base de datos actual o hasta que se cierra la conexión. Estas tablas son útiles para almacenar datos temporales que son necesarios para un procesamiento específico dentro de una sesión de base de datos. Una vez que la sesión termina, los datos de las tablas temporales se eliminan automáticamente. La sintaxis básica para crear una tabla temporal es:

```sql
CREATE TEMPORARY TABLE temp_empleados (
    SELECT * FROM tabla1
);
```

In [20]:
query_crear_tabla_temporal = '''CREATE temporary table TablaArtistAlbum AS 
                                select a2."Title" AS nombre_cancion, a."Name" AS nombre_artista 
                                FROM "Artist" a 
                                INNER JOIN "Album" a2 
                                ON a."ArtistId" = a2."AlbumId";
                            '''
ss.vistas_tablas("Chinook", "my_user", "admin", query_crear_tabla_temporal)


Conexión realizada con éxito
